1420 Assignment 1; Mohammed, Nasser
Fri, Jan 30th.

In [2]:
import numpy as np
import astropy.units as u

### 1.

Key assumptions: 

Cross section of an individual star: 1.6 x 10^-15 pc^2

Cross section of a galaxy: 3.2 x 10^8 pc^2


In [ ]:
def t_relaxation(N, t_dyn):
    """
    N: Number of particles
    t_dyn: dynamical time, in Gyr
    """
    return N/(8*np.log(N))*t_dyn*u.Gyr

def t_dynamical(r, v_c):
    return 2*np.pi*r*u.kpc/(v_c*u.km/u.s)

In [52]:
import pandas as pd
# - had chatGPT write the table into a DF
df = pd.DataFrame({
    "Object Type": ["globular cluster", "dwarf galaxy", "Milky Way", "galaxy group", "galaxy cluster"],
    "Radius": [0.01, 0.1, 10, 1000, 5000],
    "Velocity": [5, 10, 200, 200, 1000],
    "N": [10**5, 10**7, 10**11, 50, 10**3],
    "Particle": ["star", "star", "star", "galaxy", "galaxy"],})
print(df)

        Object Type   Radius  Velocity             N Particle
0  globular cluster     0.01         5        100000     star
1      dwarf galaxy     0.10        10      10000000     star
2         Milky Way    10.00       200  100000000000     star
3      galaxy group  1000.00       200            50   galaxy
4    galaxy cluster  5000.00      1000          1000   galaxy


In [63]:
df = df.copy()
df["Dynamical Time (Gyr)"] = pd.NA
for i, row in df.iterrows():
    v_c = row["Velocity"]
    r = row['Radius']
    t_dyn = t_dynamical(r, v_c)
    print(f'The dynamical time for object "{row['Object Type']}" is \t {t_dyn.to(u.Gyr):.3}')
    df.loc[i,'Dynamical Time (Gyr)'] = t_dyn.value

The dynamical time for object "globular cluster" is 	 0.0123 Gyr
The dynamical time for object "dwarf galaxy" is 	 0.0614 Gyr
The dynamical time for object "Milky Way" is 	 0.307 Gyr
The dynamical time for object "galaxy group" is 	 30.7 Gyr
The dynamical time for object "galaxy cluster" is 	 30.7 Gyr


In [64]:
df = df.copy()
df["Relaxation Time (Gyr)"] = pd.NA
for i, row in df.iterrows():
    N = row["N"]
    t_dyn = row["Dynamical Time (Gyr)"]
    t_relax =t_relaxation(N, t_dyn)
    print(f'The relaxation time for object "{row['Object Type']}" is \t {t_relax.to(u.Gyr):.3}')
    df.loc[i,'Relaxation Time (Gyr)'] = t_relax.value

The relaxation time for object "globular cluster" is 	 0.0136 Gyr
The relaxation time for object "dwarf galaxy" is 	 4.87 Gyr
The relaxation time for object "Milky Way" is 	 1.55e+05 Gyr
The relaxation time for object "galaxy group" is 	 0.0502 Gyr
The relaxation time for object "galaxy cluster" is 	 0.568 Gyr


In [65]:
df = df.copy()
df["Collisional Time (Gyr)"] = pd.NA
for i, row in df.iterrows():
    if row['Particle'] == 'star':
        cross_section = 1.6 * 10**-15 * u.pc**2
    elif row['Particle'] == 'galaxy':
        cross_section = 3.2 * 10**8 *u.pc**2

    n = row['N'] / (4/3*np.pi*(row['Radius']*u.kpc)**3) #particle density
    mean_free_path = (n*cross_section)**-1
    t_collide = mean_free_path/(row['Velocity']*u.km/u.s)

    print(f'The direction collisional time for particles in object "{row['Object Type']}" is \t {t_collide.to(u.Gyr):.3}')
    df.loc[i,'Collisional Time (Gyr)'] = t_collide.value
    


The direction collisional time for particles in object "globular cluster" is 	 5.12e+09 Gyr
The direction collisional time for particles in object "dwarf galaxy" is 	 2.56e+10 Gyr
The direction collisional time for particles in object "Milky Way" is 	 1.28e+11 Gyr
The direction collisional time for particles in object "galaxy group" is 	 1.28e+03 Gyr
The direction collisional time for particles in object "galaxy cluster" is 	 1.6e+03 Gyr


In [66]:
df

,Object Type,Radius,Velocity,N,Particle,Dynamical Time,Collisional Time,Relaxation Time,Dynamical Time (Gyr),Relaxation Time (Gyr),Collisional Time (Gyr)
0,globular cluster,0.01,5,100000,star,0.012566,5235.987756,13.643764,0.012566,13.643764,5235.987756
1,dwarf galaxy,0.10,10,10000000,star,0.062832,26179.93878,4872.772692,0.062832,4872.772692,26179.93878
2,Milky Way,10.00,200,100000000000,star,0.314159,130899.6939,155042767.482027,0.314159,155042767.482027,130899.6939
3,galaxy group,1000.00,200,50,galaxy,31.415927,0.001309,50.191305,31.415927,50.191305,0.001309
4,galaxy cluster,5000.00,1000,1000,galaxy,31.415927,0.001636,568.490147,31.415927,568.490147,0.001636


Above I've calculated the dynamical, direct collisional, and relaxation time for the provided stellar systems. The most simple divide to distinguish the dynamical state of each system is those who's relaxation time is less than the age of the universe, and those greater.

Based on the numbers provided, only the globular cluster system (GC) would reach equillibrium on order t_hubble, all other systems we cannot say are in dynamical equillibrium (not accoutning for phase-mixing or violent relaxation).

### 2.